In [1]:
import os 
import glob
import random

# load all data
npy_paths = glob.glob('/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/**/*.npy')
len(npy_paths)

121

In [2]:
# split all data to train / test & save list files

random.shuffle(npy_paths)
train_list = sorted(npy_paths[:int(len(npy_paths)*0.9)])
test_list = sorted(npy_paths[int(len(npy_paths)*0.9):])

with open('list/rlt-x3d-train.list', 'w') as f:
    for line in train_list:
        f.write(line + '\n')

with open('list/rlt-x3d-test.list', 'w') as f2:
    for line in test_list:
        f2.write(line + '\n')

In [3]:
def get_abnormal_cnt(npylist):

    abnormal = ['abnormal', 'Deceptive']
    abnormal_cnt = 0
    for nl in npylist:
        if nl.split('/')[-2] in abnormal:
            abnormal_cnt += 1
        else:
            break

    return abnormal_cnt

train_abcnt = get_abnormal_cnt(train_list)
test_abcnt = get_abnormal_cnt(test_list)

print('train normal count: ', len(train_list) - train_abcnt)
print('train abnormal count: ', train_abcnt)
print('test normal count: ', len(test_list) - test_abcnt)
print('test abnormal count: ', test_abcnt)

train normal count:  53
train abnormal count:  55
test normal count:  7
test abnormal count:  6


In [10]:
# arguments

import argparse

parser = argparse.ArgumentParser(description='RTFM')
parser.add_argument('--feat-extractor', default='i3d', choices=['i3d', 'c3d'])
parser.add_argument('--feature-size', type=int, default=2048, help='size of feature (default: 2048)')
parser.add_argument('--modality', default='RGB', help='the type of the input, RGB,AUDIO, or MIX')
parser.add_argument('--rgb-list', default='list/rlt-x3d-train.list', help='list of rgb features ')
# parser.add_argument('--test-rgb-list', default='list/rlt-x3d-test.list', help='list of test rgb features ')
# parser.add_argument('--gt', default='list/gt-ucf-x3d.npy', help='file of ground truth ')
parser.add_argument('--gpus', default=1, type=int, choices=[0], help='gpus')
parser.add_argument('--lr', type=str, default='[0.001]*15000', help='learning rates for steps(list form)')
parser.add_argument('--batch-size', type=int, default=16, help='number of instances in a batch of data (default: 16)')
parser.add_argument('--workers', default=8, help='number of workers in dataloader')
parser.add_argument('--model-name', default='rtfm_rlt_x3d_tencrop', help='name to save model')
parser.add_argument('--pretrained-ckpt', default='ckpt/rtfm_ucf_x3d_tencrop3270-x3d.pkl', help='ckpt for pretrained model')
parser.add_argument('--num-classes', type=int, default=1, help='number of class')
parser.add_argument('--dataset', default='rlt', help='dataset to train on (default: )')
parser.add_argument('--plot-freq', type=int, default=10, help='frequency of plotting (default: 10)')
parser.add_argument('--max-epoch', type=int, default=1000, help='maximum iteration to train (default: 100)')
parser.add_argument('-f')

args = parser.parse_args()

In [11]:
# train

from torch.utils.data import DataLoader
import torch.optim as optim
import torch
from utils import save_best_record
from model import Model
from dataset import Dataset
from train import train
from test_10crop import test
import option
from tqdm import tqdm
from utils import Visualizer
from config import *

# init visdom
viz = Visualizer(env='RTFM RLT X3D', use_incoming_socket=False)

config = Config(args)

# DataLoader
train_nloader = DataLoader(Dataset(args, test_mode=False, is_normal=True, abnormal_cnt=train_abcnt),
                           batch_size=args.batch_size, shuffle=True,
                           num_workers=0, pin_memory=False, drop_last=True)
train_aloader = DataLoader(Dataset(args, test_mode=False, is_normal=False, abnormal_cnt=train_abcnt),
                           batch_size=args.batch_size, shuffle=True,
                           num_workers=0, pin_memory=False, drop_last=True)
# test_loader = DataLoader(Dataset(args, test_mode=True),
#                           batch_size=1, shuffle=False,
#                           num_workers=0, pin_memory=False)

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


normal list for rlt
['/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_001.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_002.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_003.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_004.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_005.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_006.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_007.npy\n', '/home/workspaces/datasets/RealLifeTrial/Real-life_Deception_Detection_2016/features_x3d/Truthful/trial_truth_008.npy\n', '/h

In [13]:
len(train_nloader)

3

In [14]:
# load model

model = Model(args.feature_size, args.batch_size)

model.load_state_dict(torch.load(args.pretrained_ckpt))

# for name, value in model.named_parameters():
#     print(name)
print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

if not os.path.exists('./ckpt'):
    os.makedirs('./ckpt')
    

Model(
  (Aggregate): Aggregate(
    (conv_1): Sequential(
      (0): Conv1d(2048, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): Sequential(
      (0): Conv1d(2048, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      (1): ReLU()
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_3): Sequential(
      (0): Conv1d(2048, 512, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
      (1): ReLU()
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_4): Sequential(
      (0): Conv1d(2048, 512, kernel_size=(1,), stride=(1,), bias=False)
      (1): ReLU()
    )
    (conv_5): Sequential(
      (0): Conv1d(2048, 2048, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (1): ReLU()
      (2): BatchNorm1d(2048, eps=1e-

In [7]:
# args.max_epoch = 1000

In [15]:


optimizer = optim.Adam(model.parameters(),
                        lr=config.lr[0], weight_decay=0.005)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)

# test_info = {"epoch": [], "test_AUC": []}
best_AUC = -1
output_path = ''   # put your own path here
# auc = test(test_loader, model, args, viz, device)

for step in tqdm(
        range(1, args.max_epoch + 1),
        total=args.max_epoch,
        dynamic_ncols=True
):
    if step > 1 and config.lr[step - 1] != config.lr[step - 2]:
        for param_group in optimizer.param_groups:
            param_group["lr"] = config.lr[step - 1]

    if (step - 1) % len(train_nloader) == 0:
        loadern_iter = iter(train_nloader)

    if (step - 1) % len(train_aloader) == 0:
        loadera_iter = iter(train_aloader)

    train(loadern_iter, loadera_iter, model, args.batch_size, optimizer, viz, device)
    scheduler.step()
    
    if step % 100 == 0 and step > 200:

        # auc = test(test_loader, model, args, viz, device)
        # test_info["epoch"].append(step)
        # test_info["test_AUC"].append(auc)

        # if test_info["test_AUC"][-1] > best_AUC:
            # best_AUC = test_info["test_AUC"][-1]
        torch.save(model.state_dict(), './ckpt/' + args.model_name + '{}-x3d.pkl'.format(step))
            # save_best_record(test_info, os.path.join(output_path, '{}-step-AUC.txt'.format(step)))
            

torch.save(model.state_dict(), './ckpt/' + args.model_name + 'final.pkl')

100%|██████████| 1000/1000 [03:15<00:00,  5.11it/s]


In [ ]:
torch.save(model.state_dict(), './ckpt/' + args.model_name + 'final.pkl')